In [2]:
#Libraries

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import requests, bs4, os, sys
from unidecode import unidecode
from fuzzywuzzy import fuzz
from pprint import pprint
%store -r





In [5]:
############################
#####SOURCE 1: FPL DATA#####
############################

#Data Source: https://github.com/vaastav/Fantasy-Premier-League
#Originally collected using FPL player API - https://fantasy.premierleague.com/api/event/{GW}/live/
#Licence: MIT
#Data description: game week data at the player level for the past 7 premier league seasons

df_fpl_22 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2022-23/gws/merged_gw.csv')
df_fpl_21 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2021-22/gws/merged_gw.csv')
df_fpl_20 = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/merged_gw.csv')


In [6]:
#Join tables
df_full_FPL = pd.concat([df_fpl_22, df_fpl_21, df_fpl_20],sort=False, ignore_index=True)

#Checking to see if features are the same across all seasons:
missing_values = df_full_FPL.isnull().sum()
columns_with_missing_values = missing_values[missing_values > 0].index.tolist()
print("Columns that don't appear in all dataframes:\n")
print(columns_with_missing_values)


Columns that don't appear in all dataframes:

['expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded', 'starts', 'GW']


In [7]:
df_full_FPL = df_full_FPL.drop(columns=['expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded', 'starts', 'GW'])

In [9]:
df_full_FPL.isnull().sum()
df_full_FPL.dtypes
df_full_FPL[['name']]= df_full_FPL[['name']].astype(str)
df_full_FPL.rename(columns={'name': 'player_name'}, inplace=True)



In [10]:
#Just select players that have played more than 0 minutes
df_full_FPL = df_full_FPL.loc[(df_full_FPL["minutes"] > 0)]



In [55]:
#Cleaning names#

df_full_FPL['player_name'] = df_full_FPL['player_name'].str.replace('_', ' ')
df_full_FPL['player_name'] = df_full_FPL['player_name'].apply(lambda x: unidecode(x))
df_full_FPL['player_name'] = df_full_FPL['player_name'].replace(to_replace=r'\d+', value='', regex=True)
df_full_FPL['player_name'] = df_full_FPL['player_name'].replace('-', ' ')
df_full_FPL['player_name'] = df_full_FPL['player_name'].str.strip()
df_full_FPL['player_name'] = df_full_FPL['player_name'].str.lower()



In [56]:
#check shape
df_full_FPL.shape

(28644, 35)

In [57]:
df_EPL_names = df_full_FPL["player_name"].unique()
len(df_EPL_names)

928

In [ ]:
#Convert date time to date
df_full_FPL["kickoff_time"]

# Split datetime column into date and time columns
df_full_FPL[['date', 'time']] = df_full_FPL['kickoff_time'].str.split('T', expand=True)
# Drop the time column
df_full_FPL = df_full_FPL.drop('time', axis=1)
df_full_FPL = df_full_FPL.drop('kickoff_time', axis=1)

# Show the resulting DataFrame
df_full_FPL["date"]

##################
#####UNDERSTAT####
##################

#Data Source: https://understat.com/ (Webscrapping)
#Licence: NA - Data publically available and free to use.
#Data description: game week data at the player level and team level for the past 7 premier league seasons 


In [43]:
##################
####PLAYER IDS####
##################

# create url for all EPL in 2022/2023 season
base_url = 'https://understat.com/league'
league = 'EPL'
seasons = ["2022", "2021", "2020"]
ids = []

for i in seasons:  
    url = base_url+'/'+league+'/'+i
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    # Player Data listed under the <script> tags
    scripts = soup.find_all('script')
    string_with_json_obj = ''

    # Find data for playerrs
    for el in scripts:
        if 'playersData' in el.text:
            string_with_json_obj = el.text.strip()


    # print(string_with_json_obj)

    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]

    json_data = json_data.encode('utf8').decode('unicode_escape')

    # convert JSON data into Python dictionary
    data = json.loads(json_data)

    for item in data:
        ids.append(item['id'])

ids = set(ids) #To make them unique
ids = list(ids) #Then back to a list to make it scriptable
len(ids)

887

In [44]:
print(ids)

['482', '10072', '8180', '7240', '585', '2245', '1621', '5553', '986', '775', '3298', '593', '6088', '3729', '8865', '725', '978', '5646', '10004', '1711', '7701', '7698', '592', '885', '966', '11208', '647', '7322', '486', '695', '7007', '8965', '2199', '790', '7891', '11000', '7700', '10761', '644', '594', '9287', '3697', '11007', '9284', '10001', '10864', '6859', '5043', '5191', '8222', '3294', '5989', '1750', '9499', '6108', '1389', '770', '5603', '9746', '6062', '11094', '2251', '951', '528', '835', '2496', '6310', '749', '1013', '6523', '7707', '8723', '87', '638', '2498', '1747', '2383', '453', '759', '6163', '9676', '9332', '6050', '6857', '6840', '789', '876', '2266', '8966', '5220', '1801', '6834', '702', '618', '1828', '2164', '844', '181', '4490', '10866', '699', '6888', '8992', '6418', '7153', '8582', '3683', '10405', '7990', '3231', '10757', '2310', '9912', '6756', '7338', '500', '7991', '842', '6893', '2385', '5232', '8045', '1776', '646', '11359', '9735', '750', '8864',

In [45]:
#Get player name in a list

# create urls for each individual player
base_url = 'https://understat.com/player'
players = ids

# Iterate through all 887 player ids
ids=list(map(int, ids))
list_names=[]
for i in range(len(ids)):
    url = base_url+'/'+str(players[i])
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    player_name = soup.find_all('div', class_="header-wrapper")
    for name in player_name:
        list_names.append((name.text.strip()))

len(list_names) #Check length is the same as before

887

In [46]:
print(list_names)

['Roberto Firmino', 'Samuel Edozie', 'Owen Otasowie', 'Benoit Badiashile Mukinayi', 'Seamus Coleman', 'Raphael Varane', 'Marcos Alonso', 'Declan Rice', 'Danny Ings', 'Andros Townsend', 'Layvin Kurzawa', 'Aaron Lennon', 'Orel Mangala', 'Hassane Kamara', 'Ollie Watkins', 'Ola Aina', 'Sam Johnstone', 'Cafú', 'Romeo Lavia', 'Adam Forshaw', 'Moussa Djenepo', 'Leandro Trossard', 'Ross Barkley', 'Kyle Walker-Peters', 'Alex Oxlade-Chamberlain', 'Mateo Joseph', 'Harry Kane', 'Bukayo Saka', 'Adam Lallana', 'Bertrand Traoré', 'Duje Caleta-Car', 'Vladimir Coufal', 'Pablo Sarabia', 'Nathan Redmond', 'Renan Lodi', 'Matheus Nunes', 'Che Adams', 'Alex Mighten', 'Erik Lamela', 'Romelu Lukaku', 'Tyler Onyango', 'Odsonne Edouard', 'Fábio Vieira', 'Jakub Moder', 'Stuart McKinstry', 'Marcos Senesi', 'Kortney Hause', 'Kiko Femenía', 'Álvaro Fernández', 'Japhet Tanganga', 'Edinson Cavani', 'Moussa Niakhate', 'Ainsley Maitland-Niles', 'Cody Drameh', 'Rodrigo Bentancur', 'Jorginho', 'Ayoze Pérez', 'Aaron Ramsd

In [47]:
##################
####PLAYER INFO###
##################

base_url = 'https://understat.com/player'
players = ids
columns = ['goals', 'shots', 'xG', 'time', 'position', 'h_team', 'a_team',
       'h_goals', 'a_goals', 'date', 'id', 'season', 'roster_id', 'xA',
       'assists', 'key_passes', 'npg', 'npxG', 'xGChain', 'xGBuildup',
       'player_id', 'player_name']

df_all = pd.DataFrame(columns=columns)
for i in range(len(ids)):
    url = base_url+'/'+str(players[i])
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    scripts = soup.find_all('script')
    string_with_json_obj = ''

    # Find data for teams
    for el in scripts:
        if 'matchesData' in el.text:
          string_with_json_obj = el.text.strip()
      
    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    # convert JSON data into Python dictionary
    new_data = json.loads(json_data)
    df_name = 'df_' + str(i)
    df_name = pd.DataFrame(new_data)
    df_name['player_id'] = str(ids[i])
    df_name['player_name'] = str(list_names[i])
    df_all = pd.concat([df_all, df_name], ignore_index=True)


In [49]:
####SET COLUMN TYPES###
df_all[['time', 'goals','shots','h_goals','a_goals','id','season','roster_id','assists','key_passes','npg','player_id']] = df_all[['time', 'goals','shots','h_goals','a_goals','id','season','roster_id','assists','key_passes','npg','player_id']].astype(int)
df_all[['xG','xA', 'npxG', 'xGChain', 'xGBuildup']] = df_all[['xG','xA', 'npxG', 'xGChain', 'xGBuildup']].astype(float)
df_all[['player_name', 'position', 'h_team', 'a_team']]= df_all[['player_name', 'position', 'h_team', 'a_team']].astype(str)
df_all[['time', 'goals','shots','h_goals','a_goals','id','season','roster_id','assists','key_passes','npg','player_id']] = df_all[['time', 'goals','shots','h_goals','a_goals','id','season','roster_id','assists','key_passes','npg','player_id']].round(1)

In [54]:
#cleaning names in df_all[df_all["player_name"]
df_all['player_name'] = df_all['player_name'].apply(lambda x: unidecode(x))
df_all['player_name'] = df_all['player_name'].replace(to_replace=r'\d+', value='', regex=True)
df_all['player_name'] = df_all['player_name'].replace('-', ' ')
df_all['player_name'] = df_all['player_name'].str.strip()
df_all['player_name'] = df_all['player_name'].str.lower()


In [59]:
#Filter just for EPL teams and seasons.
EPL_teams_2016_2022 = ['Southampton', 'Bournemouth', 'Chelsea', 'Newcastle United', 'Leicester',
       "Nottingham Forest", 'Crystal Palace', 'Wolverhampton Wanderers', 'Brentford', 'Tottenham',
       'West Ham', 'Liverpool', 'Leeds', 'Fulham', 'Brighton', 'Manchester City',
       'Manchester Utd', 'Everton', 'Arsenal', 'Aston Villa', 'Norwich', 'Watford', 'Burnley',
       'West Bromwich Albion', 'Sheffield United']

# Check if values from the list exist in h_team or a_team columns
EPL_teams = df_all['h_team'].isin(EPL_teams_2016_2022) | df_all['a_team'].isin(EPL_teams_2016_2022)

# Filter the rows in the dataframe based on the mask
df_all = df_all[EPL_teams & (df_all['date'] >= '2020-09-12') & (df_all['time'] >0)]
df_all.shape

# 28346 vs 28644, so there is a discrepency somewhere.

(28346, 22)

In [60]:
%store df_all
%store df_full_FPL


Stored 'df_all' (DataFrame)
Stored 'df_full_FPL' (DataFrame)


Now going to join tables on name and date to see if they are mismatched. 
If they are I will correct the dates

In [62]:
#Merge tables on name and date
#First re name name as player_name in FPL table

# merge the tables on player_name and date
merged_table = pd.merge(df_full_FPL_test, df_all_test, on=['player_name', 'date'], how='outer', indicator=True)

# display the rows that didn't match
unmatched_rows = merged_table[merged_table['_merge'] != 'both']
unmatched_rows[["player_name", "date"]]

,player_name,date


In [63]:
#Luckily only one match had an incorrect date. It was out by 1 day. This has now been edited and the tables match.
df_all_test.loc[(df_all_test['h_team'] == 'Arsenal') & (df_all_test['a_team'] == 'Everton') & (df_all_test['date'] == '2023-03-02'), 'date'] = '2023-03-01'

In [15]:
#Saving table
#merged_table.to_csv("merged_table.csv", index=False)
merged_table = pd.read_csv('/Users/jon/Documents/fpl_points_prediction/python exports/merged_table.csv', index_col=False)


In [17]:
#Renaming columns with same names
merged_table = merged_table.rename(columns={'position_x': 'position', 'assists_y': 'assists'})

In [18]:
#Dropping duplicate columns
merged_table = merged_table.drop(['element', 'position_y', 'assists_x'], axis=1)
merged_table = merged_table.drop(['goals_scored', 'fixture', 'transfers_balance', 'time', 'roster_id', 'id', 'h_goals', 'a_goals'], axis=1)
merged_table = merged_table.drop(['_merge'], axis=1)



In [ ]:
#Check and remove duplicate rows
before = len(merged_table)
data = merged_table.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

In [20]:
merged_table_clean = merged_table
merged_table_clean.to_csv("merged_table_clean.csv", index=False)


In [21]:
%store merged_table_clean

Stored 'merged_table_clean' (DataFrame)


In [ ]:
#########################
####538 - SOCCOR API####
#########################

seasons_num = [2022, 2021, 2020]

import pandas as pd

URL = 'https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv'
data_538 = pd.read_csv(URL)
PL = data_538[(data_538['season'].isin(seasons_num)) & (data_538['league'] == 'Barclays Premier League')]

In [ ]:
#Renaming teams

PL = PL.rename(columns={'team1': 'h_team', 'team2': 'a_team'})

PL.loc[PL['h_team'] == 'Leeds United', 'h_team'] = 'Leeds'
PL.loc[PL['a_team'] == 'Leeds United', 'a_team'] = 'Leeds'
PL.loc[PL['h_team'] == 'West Ham United', 'h_team'] = 'West Ham'
PL.loc[PL['a_team'] == 'West Ham United', 'a_team'] = 'West Ham'
PL.loc[PL['h_team'] == 'Wolverhampton', 'h_team'] = 'Wolverhampton Wanderers'
PL.loc[PL['a_team'] == 'Wolverhampton', 'a_team'] = 'Wolverhampton Wanderers'
PL.loc[PL['h_team'] == 'Tottenham Hotspur', 'h_team'] = 'Tottenham'
PL.loc[PL['a_team'] == 'Tottenham Hotspur', 'a_team'] = 'Tottenham'
PL.loc[PL['h_team'] == 'Brighton and Hove Albion', 'h_team'] = 'Brighton'
PL.loc[PL['a_team'] == 'Brighton and Hove Albion', 'a_team'] = 'Brighton'
PL.loc[PL['h_team'] == 'Leicester City', 'h_team'] = 'Leicester'
PL.loc[PL['a_team'] == 'Leicester City', 'a_team'] = 'Leicester'
PL.loc[PL['h_team'] == 'Leeds United', 'h_team'] = 'Leeds'
PL.loc[PL['a_team'] == 'Leeds United', 'a_team'] = 'Leeds'
PL.loc[PL['h_team'] == 'AFC Bournemouth', 'h_team'] = 'Bournemouth'
PL.loc[PL['a_team'] == 'AFC Bournemouth', 'a_team'] = 'Bournemouth'
PL.loc[PL['h_team'] == 'Norwich City', 'h_team'] = 'Norwich'
PL.loc[PL['a_team'] == 'Norwich City', 'a_team'] = 'Norwich'
PL.loc[PL['h_team'] == 'Newcastle', 'h_team'] = 'Newcastle United'
PL.loc[PL['a_team'] == 'Newcastle', 'a_team'] = 'Newcastle United'

PL["h_team"].unique()
#PL.to_csv("match_data.csv", index=False)

In [ ]:
#Check for missing data
#match_data = pd.read_csv('/Users/jon/Documents/fpl_points_prediction/ERD/match_data.csv', delimiter=";", index_col=False)
null_cols = PL.isnull().sum()
null_cols[null_cols > 0]

null_importance_h = PL[(PL['importance_h'].isnull()==True)]
null_importance_h = null_importance_h[['season_id','date','h_team_id',	'a_team_id','importance_h', 'importance_a']]
null_importance_h


In [ ]:
null_importance_h = PL[(PL['h_score'].isnull()==True)]
null_importance_h = null_importance_h[['season_id','date','h_team_id',	'a_team_id', 'h_score', 'a_score', 'h_xg', 'a_xg', 'h_nsxg', 'a_nsxg'  ]]
null_importance_h

In [ ]:
#For important_h and importance_a for the first 2 missing values I chose to fill with importance for the season before for the same game as they were played at a similar time of the year.
#For the last match, the games in either season were played at different times and so the importance would be different. 
    #In this case I took an average of the teams importance 2 games either side of the date.

In [ ]:
# merge with other datasets on team1, team2, season to check that tables are consistent
spi_merged_table = pd.merge(merged_table_clean, PL, on=['h_team', 'a_team', 'season'], how='inner', indicator=True)

# display the rows that didn't match
unmatched_rows = spi_merged_table[spi_merged_table['_merge'] != 'both']
unmatched_rows[['h_team', 'a_team', 'season']]

#Tables are consistent and so can be merged and saved as one table.
spi_merged_table.columns
spi_merged_table.to_csv("spi_merged_table.csv", index=False)